<a href="https://colab.research.google.com/github/jfjoung/AI_For_Chemistry/blob/main/notebooks/week10/Week_10_One_step_retrosynthesis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
